In [396]:
from sys import path
path.insert(0,'../python')
import numpy as np
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import unicodedata
import json
from googlemaps import Client
from yaml import load as yaml_load, FullLoader
from google_api import *
import folium
from folium.plugins import MarkerCluster,HeatMap, Fullscreen
from async_request import make_request

# Obtenemos la lista de pueblos mágicos

In [2]:
url = 'https://www.gob.mx/sectur/articulos/pueblos-magicos-206528'

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = get(url, headers= headers)
response.status_code

200

In [4]:
content = unicodedata.normalize("NFKD", response.content.decode('utf-8'))
soup = BeautifulSoup(content, 'html.parser')
#print(soup.prettify())

In [5]:
ol = soup.find_all('ol')[1]
pueblos_text = [li.get_text() for li in ol.find_all('li')]

In [7]:
pueblos = []
for i, pueblo in enumerate(pueblos_text):
    pueblos.append({'name':pueblo})

# with open('../data/pueblos_magicos.json', 'w') as outfile:
#     json.dump(pueblos, outfile)
len(pueblos)

121

# Obtención de las coordenadas de los pueblos mágicos

In [11]:
yaml_file = yaml_load(open('../auth/google.yaml'), Loader=FullLoader)
GOOGLE_API_KEY = yaml_file['GOOGLE_API_KEY']
gmaps = Client(key=GOOGLE_API_KEY, queries_per_second = 500)

In [12]:
with open('../data/pueblos_magicos.json') as json_file:
    pueblos = json.load(json_file)

### Opción 1

```python
coordenadas = []
for pueblo in pueblos:
    geocode_result = gmaps.geocode(pueblo['name'] + ' Mexico')
    coords = get_geocode_locations(geocode_result)[0]
    coordenadas.append(coords)
    
```

### Opción 2

```python
from async_request import make_request

def fetch_geocode(session, data, i):
        try:
            response = gmaps.geocode(data)
            return response
        except Exception as e:
            print('Error trying to request the directions', e)
            return []
        
data = [pueblo['name'] + ', Mexico' for pueblo in pueblos]

geocode_results = make_request(fetch_geocode, data) 

coordinates = [get_geocode_locations(geocode_result)[0] for geocode_result in geocode_results]

for i,pueblo in enumerate(pueblos):
    pueblo['coords'] = coordinates[i]
    
```

In [14]:
# with open('../data/pueblos_magicos.json', 'w') as outfile:
#     json.dump(pueblos, outfile)

## Visualización de los pueblos en el mapa

In [15]:
with open('../data/pueblos_magicos.json') as json_file:
    pueblos = json.load(json_file)

In [16]:
tiles_url = 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'
attrb ='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
m = folium.Map(tiles=None)

folium.TileLayer(tiles_url,attr=attrb,name = 'Pueblos Mágicos de México', show = True).add_to(m)

text = '<h4>Planta Bosch Aguascalientes</h4>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)


pt_lyr = folium.FeatureGroup(name = 'Pueblos Mágicos',show=True)

for pueblo in pueblos:
    point = pueblo['coords']
    name = pueblo['name']
    text = f'<h4>{name}</h4>'
    text = folium.Html(text, script=True)
    popup = folium.Popup(text, max_width=2650)

    folium.CircleMarker(point,radius = 4,
                    popup = popup,
                    fill=True, # Set fill to True|
                    fill_color='white',
                    color = 'green',
                    fill_opacity=1).add_to(pt_lyr)
pt_lyr.add_to(m)

Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds(m.get_bounds())
m
#m.save('../maps/Mapeo Asociados Aguascalientes.html')

## Obtención de la matriz de distancia y tiempo

In [17]:
coordinates = [pueblo['coords'] for pueblo in pueblos]

In [18]:
origins = [coordinates[1]]
center = len(coordinates) // 2
destinations1 = coordinates[:center]
destinations2 = coordinates[center:]

In [19]:
distance_matrix_result1 = gmaps.distance_matrix(origins, destinations1, avoid = None) #avoid = 'ferries'
distance_matrix_result2 = gmaps.distance_matrix(origins, destinations2, avoid = None) #avoid = 'ferries'

In [20]:
distances = get_distance_matrix(distance_matrix_result1)[0] + get_distance_matrix(distance_matrix_result2)[0]
durations = distances = get_distance_matrix(distance_matrix_result1)[0] + get_distance_matrix(distance_matrix_result2)[0]

In [21]:
def fetch_distance_matrix(session, data, i):
        origins, destinations = data

        try:
            center = len(destinations) // 2
            destinations1 = destinations[:center]
            destinations2 = destinations[center:]
            distance_matrix_result1 = gmaps.distance_matrix(origins, 
                                                            destinations1, 
                                                            avoid = None) #avoid = 'ferries'
            distance_matrix_result2 = gmaps.distance_matrix(origins, 
                                                            destinations2, 
                                                            avoid = None) #avoid = 'ferries'
            return [distance_matrix_result1, distance_matrix_result2]
        except Exception as e:
            print('Error trying to request the directions', e)
            return []

```python
destinations = coordinates
data = ([[coord], destinations] for coord in coordinates)
distance_matrix_results = make_request(fetch_distance_matrix, data) 
```

```python
distance_matrix = []
duration_matrix = []
for distance_matrix_result1, distance_matrix_result2 in distance_matrix_results:
    distances = get_distance_matrix(distance_matrix_result1)[0] + get_distance_matrix(distance_matrix_result2)[0]
    durations = get_duration_matrix(distance_matrix_result1)[0] + get_duration_matrix(distance_matrix_result2)[0]
    distance_matrix.append(distances)
    duration_matrix.append(durations)
    
    
distance_matrix_df = pd.DataFrame(distance_matrix)
duration_matrix_df = pd.DataFrame(duration_matrix)

distance_matrix_df.to_csv('../data/distance_matrix.csv')
duration_matrix_df.to_csv('../data/duration_matrix.csv')

```

In [51]:
distance_matrix = pd.read_csv('../data/distance_matrix.csv', index_col=0)
duration_matrix = pd.read_csv('../data/duration_matrix.csv', index_col=0)

In [52]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,0,1498723,40500,369772,707400,273755,1442476,1481054,881684,91402,...,154472,958852,89431,405029,252000,327487,224798,258754,154424,373429
1,1524409,0,1488006,1678246,1388937,1770922,2939643,406117,1550430,1501303,...,1598188,1192283,1545317,1902197,1749167,1714250,1384156,1755921,1572015,1870596
2,39952,1453417,0,350247,687875,321247,1489969,1454196,862159,82489,...,189830,939326,136924,452522,299493,307962,185611,306247,153231,420922
3,370238,1635377,351123,0,633055,467349,1611286,1544693,686879,259771,...,516149,884506,447517,557661,389168,43666,449821,449004,225168,380167
4,708325,1344824,689209,633649,0,969927,2138648,912923,182911,689687,...,854236,252736,785603,1101202,948173,669654,695279,954926,760398,875486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,327953,1671381,308837,43666,669059,589555,1563697,1580698,722884,217486,...,473864,920511,405231,720830,341579,0,407536,397415,211203,332578
117,222667,1358793,193251,448459,693604,466173,1634894,1380741,867888,206548,...,293440,848211,240568,597448,444419,406173,0,451172,277260,565848
118,257850,1730055,305041,451108,952516,158337,1334528,1726169,973802,336518,...,328580,1203967,246335,248693,78584,364828,448972,0,240625,94450
119,154505,1545938,152865,225562,755187,291069,1459790,1528840,929471,73071,...,300416,1006638,164253,422344,234054,211597,278132,240808,0,355483


## Creación del grafo

In [104]:
import networkx as nx

In [105]:
G = nx.from_numpy_matrix(np.asarray(distance_matrix))

In [106]:
import acopy

In [107]:
solver = acopy.Solver(rho=.04, q=1, top = 5)
colony = acopy.Colony(alpha=1, beta=1)

In [108]:
tour = solver.solve(G, colony, limit=200)

In [117]:
solution = {'cost': tour.cost,
 'path': tour.path,
 'node':tour.nodes}

In [119]:
# with open('../data/solution1.json', 'w') as outfile:
#     json.dump(solution, outfile)

In [120]:
with open('../data/solution1.json') as json_file:
    tour = json.load(json_file)

In [145]:
solution['cost'] / 1000

28850.619

In [125]:
path = np.asarray(coordinates)[tour['node']]

In [126]:
tiles_url = 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'
attrb ='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
m = folium.Map(tiles=None)

folium.TileLayer(tiles_url,attr=attrb,name = 'Pueblos Mágicos de México', show = True).add_to(m)

text = '<h4>Planta Bosch Aguascalientes</h4>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)


pt_lyr = folium.FeatureGroup(name = 'Pueblos Mágicos',show=True)


folium.PolyLine(path).add_to(pt_lyr)
pt_lyr.add_to(m)

Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds(m.get_bounds())
m
#m.save('../maps/Mapeo Asociados Aguascalientes.html')

In [209]:
class Graph:
    
    def __init__(self, coords, edges = None):
        
        self.n = len(coords)
        self.node = [Node(point) for point in coords]
        if edges is None:
            self.edges = np.zeros([self.n ,self.n ])
            for i in range(self.n):
                for j in range(self.n):
                    self.edges[i,j] = calculateDistance(self.node[i], self.node[j])
        else:
            self.edges = edges
class Node:
    
    def __init__(self, point):
        self.x = point[0]
        self.y = point[1]
        
    def __str__(self):
        return str([self.x, self.y])

def calculateDistance(n1,n2):
    return np.sqrt((n1.x - n2.x)**2 + (n1.y - n2.y)**2)

class Colony:
    
    def __init__(self, graph, antNo, tau, eta, alpha, beta):
        
        self.queen = Ant()
        self.ant = [Ant() for i in range(antNo)]
        
        nodeNo = graph.n
    
        for i in range(antNo):
            initial_node = np.random.randint(nodeNo)
            self.ant[i].tour[0] = initial_node
            for j in range(1, nodeNo):
                currentNode = self.ant[i].tour[-1]
                P_allNodes = tau[currentNode,:]**alpha * eta[currentNode,:]**beta
                P_allNodes[self.ant[i].tour] = 0
                P = P_allNodes / np.sum(P_allNodes)
                nextNode = rouletteWheel(P)
                self.ant[i].tour.append(nextNode)
            #complete the tour
            self.ant[i].tour.append(self.ant[i].tour[0])

class Ant:
    
    def __init__(self):
        self.fitness = 0
        self.tour = [None]
        
        
def rouletteWheel(P):
    cumSumP = np.cumsum(P)
    r = np.random.random()
    nextNode = np.where(r <= cumSumP)[0]
    return nextNode[0]

def fitnessFunction(tour, graph):
    fitness = 0
    
    for i in range(len(tour) - 1):
        currentNode = tour[i]
        nextNode = tour[i+1]
        fitness += graph.edges[currentNode, nextNode]
    return fitness

def updatePhromone(tau, colony):
    
    nodeNo = len(colony.ant[1].tour)
    antNo = len(colony.ant)
    
    for i in range(antNo):
        for j in range(nodeNo - 1):
            currentNode = colony.ant[i].tour[j]
            nextNode = colony.ant[i].tour[j+1]
            tau[currentNode, nextNode] = tau[currentNode, nextNode] + 1 / colony.ant[i].fitness
            
    return tau

In [426]:
## Creating the graph
graph = Graph(coordinates, np.asarray(distance_matrix))

## Initial parameters for ACO

maxIter = 500
antNo = 25
k = 1
rho = 0.5 #evaporation rate
alpha = 1 #Phromone exponential parameters
beta = 1 #Desiarability exponential parameter


bestFitness = np.inf 
bestTour = None
tau0 =  k /(graph.n * np.mean(graph.edges)) #Initial phromone concentration
tau = tau0*np.ones([graph.n ,graph.n ]) #phromone matrix
with np.errstate(divide='ignore'):
    eta = 1 / graph.edges #desiarability of each edge
    


In [427]:
for i in range(maxIter):
    #creat ants
    colony = Colony( graph, antNo, tau, eta, alpha, beta)

    #Calculate the fitness values of all ants
    for ant in colony.ant:
        ant.fitness = fitnessFunction(ant.tour, graph)

    allAntsFitness = np.asarray([ant.fitness for ant in colony.ant])

    minIndex = allAntsFitness.argmin()
    minVal = allAntsFitness[minIndex]
    if minVal < bestFitness:
        bestFitness = minVal
        bestTour = colony.ant[minIndex].tour

    colony.queen.tour = bestTour
    colony.queen.fitness = bestFitness

    #Update phromone matrix

    tau = updatePhromone(tau, colony)
    
    #Evaporation 
    
    tau = (1-rho)*tau

In [428]:
bestFitness

20018584

In [431]:
path = np.asarray(coordinates)[bestTour]

In [432]:
solution = {'cost': int(bestFitness),
            'tour': [int(x) for x in bestTour]
           }

In [433]:
solution['cost']

20018584

In [434]:
# with open('../data/solution2.json', 'w') as outfile:
#     json.dump(solution, outfile)

In [435]:
with open('../data/solution2.json') as json_file:
    tour = json.load(json_file)

In [436]:
path = np.asarray(coordinates)[tour['tour']]

In [437]:
tiles_url = 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'
attrb ='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
m = folium.Map(tiles=None)

folium.TileLayer(tiles_url,attr=attrb,name = 'Pueblos Mágicos de México', show = True).add_to(m)

text = '<h4>Planta Bosch Aguascalientes</h4>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)


pt_lyr = folium.FeatureGroup(name = 'Pueblos Mágicos',show=True)


folium.PolyLine(path).add_to(pt_lyr)
pt_lyr.add_to(m)

Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds(m.get_bounds())
m
#m.save('../maps/Mapeo Asociados Aguascalientes.html')

In [438]:
paths = [[path[i-1], path[i]]for i in range(1, len(path))] 
origin, destintaion = paths[0]

In [439]:
result = gmaps.directions(origin, destintaion,  avoid =  None)

In [440]:
polyline = get_directions_polylines(result)[0]

```python
def fetch_directions(session, data, i):
        try:
            origin, destintaion = data
            response = gmaps.directions(origin, destintaion,  avoid =  None)
            return response
        except Exception as e:
            print('Error trying to request the directions', e)
            return []
data = paths
directions_results = make_request(fetch_directions, data)

with open('../data/path_directions_result2.json', 'w') as outfile:
    json.dump(directions_results, outfile)
    
```

In [442]:
with open('../data/path_directions_result2.json') as json_file:
    directions_results = json.load(json_file)

In [443]:
sum([get_directions_total_distance(directions_result)[0] for directions_result in directions_results])

20018593

In [444]:
polylines = [get_directions_polylines(directions_result)[0] for directions_result in directions_results]

In [445]:
tiles_url = 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'
attrb ='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
m = folium.Map(tiles=None)

folium.TileLayer(tiles_url,attr=attrb,name = 'Pueblos Mágicos de México', show = True).add_to(m)

text = '<h4>Planta Bosch Aguascalientes</h4>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)


pt_lyr = folium.FeatureGroup(name = 'Pueblos Mágicos',show=True)

for polyline in polylines:
    folium.PolyLine(decode_polyline(polyline), weight = 5).add_to(pt_lyr)
    
    
for pueblo in pueblos:
    point = pueblo['coords']
    name = pueblo['name']
    text = f'<h4>{name}</h4>'
    text = folium.Html(text, script=True)
    popup = folium.Popup(text, max_width=2650)

    folium.CircleMarker(point,radius = 4,
                    weight = 3,
                    popup = popup,
                    fill=True, # Set fill to True|
                    fill_color='white',
                    color = 'red',
                    fill_opacity=1).add_to(pt_lyr)
    
pt_lyr.add_to(m)

Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds(m.get_bounds())
m
#m.save('../maps/Mapeo Asociados Aguascalientes.html')